In [129]:
import pandas as pd
import json
import matplotlib.pyplot as plot
import time
import os
import csv
import requests
from TwitterCaller import *
from DiscordCaller import *
from TelegramCaller import *


In [130]:

SLUG_URL = "https://api.opensea.io/api/v1/collection/{}"
BASE_URL_COLLECTION = "https://opensea.io/collection/{}"

telegramCaller = TelegramCaller()


In [131]:
def keys(d, c = []):
      return [i for a, b in d.items() for i in ([c+[a]] if not isinstance(b, dict) else keys(b, c+[a]))]

In [132]:
def get_slug_json(slug_names):
    try:
        slug_dict = dict()
        for slug in slug_names:
            response = requests.request("GET", SLUG_URL.format(slug))
            if response.text:
                slug_dict.update({slug:response.text})
            else:
                slug_dict.update({slug:None})
        with open('slug_info.json','w') as f:
            json.dump(slug_dict, f, indent = 6)
    except Exception as e:
        print("An error occurred",e)

In [133]:
def get_slug_info(slug_name):
    try:   
        social_dict = dict()
        with open('slug_info.json','r') as f:
            response = json.loads(f.read())
            if response[slug_name]:
                response_body = json.loads(response[slug_name])
                objectKeys = list(map('.'.join, keys(response_body)))
                if 'collection.twitter_username' in objectKeys:
                    twitter_username = response_body['collection']['twitter_username']
                    if twitter_username:
                        social_dict.update(**get_user_info(response_body['collection']['twitter_username']))
                if 'collection.discord_url' in objectKeys:
                    discord_invite_link = response_body['collection']['discord_url']
                    if discord_invite_link:
                        social_dict.update(**get_discord_info(response_body['collection']['discord_url']))

            return social_dict
    except Exception as e:
        print("An error occurred while getting social data",e)
        return dict()

In [134]:
def get_stats(slug_name,param):
    try:
        with open('slug_info.json','r') as f:
            response = json.loads(f.read())
            if response[slug_name]:
                response_body = json.loads(response[slug_name])
                objectKeys = list(map('.'.join, keys(response_body)))
                if 'collection.stats.{}'.format(param) in objectKeys:
                    return response_body['collection']['stats'][param]
    except Exception as e:
        print("An error occurred while executing get_floor_info",e)

In [135]:
def generate_message(slug_name, dataframe):
    try:
        data_name = dataframe[dataframe['Slug'] == slug_name]
        message = '📛 {slug_name}: \n ' \
            '💸 Floor : {Floor} \n' \
            '✔️ One Day Sales : {One_day_sales} \n'\
            '💰 Seven Day Average Price : {Seven_Day_Average_Price}'.format(slug_name=slug_name,
                                                                           Floor=data_name['Floor'].iloc[0],
                                                                           One_day_sales=data_name[
                                                                               'One_Day_Sales'].iloc[0],
                                                                           Seven_Day_Average_Price=data_name['Seven_Day_Average_Price'].iloc[0])
        social_message = social_info_to_message(data_name['Social'].iloc[0])
        if social_message:
            message += social_message
            message += '\n' + BASE_URL_COLLECTION.format(slug_name)

        return message
    except Exception as e:
        print("An error occurred while generating message", e)


In [136]:
def social_info_to_message(socialObj):
    try:
        
        discord_message = "🎤 Discord : "
        twitter_message = "🐦 Twitter : "
        social_message = ' \n{}\n{}'
        
        objectKeys = list(map('.'.join, keys(socialObj)))
        
        if 'MemberCount' in objectKeys:
            discord_message += " \n🎤 Total Members : {}".format(socialObj['MemberCount'])
        if 'OnlineCount' in objectKeys:
            discord_message += " \n🎤 Online Members : {}".format(socialObj['OnlineCount'])
        
        if 'follower' in objectKeys:
            twitter_message += " \nFollower On Twitter : {}".format(socialObj['follower'])
        if 'tweet' in objectKeys:
            twitter_message += " \nTotal Tweets: {}".format(socialObj['tweet'])
        
        return social_message.format(twitter_message,discord_message) if socialObj else {}
    except Exception as e:
        print("An error occurred while generating formatted Social Info",e)
        return dict()

In [137]:
def process_data():
    nftNamesList = list()
    jsonTimeFrameArray = list()
    file = open("data.txt", "r")

    for row in file.read().split('\n'):
        if row:
            jsonTimeFrameArray.append(json.loads(row))

    rawEdgesData = list()
    for x in jsonTimeFrameArray:
        if 'assetEvents' in x['data'].keys():
            rawEdgesData.append(x['data']['assetEvents']['edges'])

    for y in rawEdgesData:
        for xy in y : 
            objectKeys = list(map('.'.join, keys(xy)))
            if 'node.assetQuantity.asset.collection.name' in objectKeys:
                collection = xy['node']['assetQuantity']['asset']['collection']
                nftCollectionDict = dict()
                nftCollectionDict = {'Slug': collection['slug'],
                                    'Name':collection['name']}
                nftNamesList.append(nftCollectionDict)
    
    if nftNamesList:
        
        df = pd.DataFrame(nftNamesList)

        df = df.Slug.value_counts().reset_index().rename(
               columns={'index': 'Slug', 'Slug': 'Count'})
        df = df[df["Count"] > 10]

        get_slug_json(df['Slug'].tolist())

        df['Social'] = df['Slug'].map(get_slug_info)
        df['Floor'] = df['Slug'].apply(lambda x : get_stats(x,'floor_price'))
        df['One_Day_Sales'] = df['Slug'].apply(lambda x : get_stats(x,'one_day_sales'))
        df['One_Day_Volume'] = df['Slug'].apply(lambda x : get_stats(x,'one_day_volume'))
        df['Seven_Day_Average_Price'] = df['Slug'].apply(lambda x : get_stats(x,'seven_day_average_price'))
        df['Market_Cap'] = df['Slug'].apply(lambda x : get_stats(x,'market_cap'))

        df.to_csv('report_600sec.csv', index=False, quotechar='"',
                            header=True, quoting=csv.QUOTE_NONE,sep=";")
        try:
            if df['Slug'].tolist():
                df.plot(x ='Slug', y='Count', kind = 'bar')
                plot.savefig('chart.png', bbox_inches = 'tight')
                telegramCaller.send_photo('chart.png')
        except Exception as e:
            time.sleep(1)
            print("An error occurred while plotting and sending data",e)
        
        df['Message'] = df['Slug'].map(lambda x : generate_message(x,df))
        
        messages = df['Message'].to_list()
        for message in messages:
            telegramCaller.send_message(message)

In [138]:
while True:
    process_data()
    #file = open("data.txt","r+")
    #file.truncate(0)
    #file.close()
    time.sleep(150)